In [12]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
from keras.layers import Input, concatenate, add, Multiply, Lambda
from keras.models import Model
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Model
from scipy.ndimage import gaussian_filter
from scipy.ndimage import distance_transform_edt, grey_closing, binary_closing, gaussian_filter
from pathlib import Path
import os
import cv2
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sys
from mpl_toolkits.axes_grid1 import make_axes_locatable
sys.path.append(os.path.abspath("/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Models_Arch"))
sys.path.append(os.path.abspath("/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork"))
from mask_loss import MaskLoss
from mask_loss import MAELoss
# print(str(Path(__file__).parent))
from ResidualUnet import Residual_Unet
from Unet import Unet
from Unet_7Kernel import Unet_7Kernel
from Unet_5Kernel import Unet_5Kernel
from Unet_3Dense import Unet_3Dense
from Unet_1Dense import Unet_1Dense
from Unet_2Dense import Unet_2Dense
from Unet_1Dense_7Kernel import Unet_1Dense_7Kernel
from Unet_1Dense_5Kernel import Unet_1Dense_5Kernel
from Unet_2Dense_7Kernel import Unet_2Dense_7Kernel
from Unet_2Dense_5Kernel import Unet_2Dense_5Kernel
from Unet_3Dense_7Kernel import Unet_3Dense_7Kernel
from Unet_3Dense_5Kernel import Unet_3Dense_5Kernel
from ResidualUnet_1Dense import Residual_Unet_1D
from ResidualUnet_2Dense import Residual_Unet_2D
from ResidualUnet_3Dense import Residual_Unet_3D
from ResidualUnet_1Dense_7Kernels import Residual_Unet_1D_7K
from ResidualUnet_1Dense_5Kernels import Residual_Unet_1D_5K
from ResidualUnet_2Dense_7Kernels import Residual_Unet_2D_7K
from ResidualUner_2Dense_5Kernels import Residual_Unet_2D_5K
from ResidualUnet_3Dense_7Kernels import Residual_Unet_3D_7K
from ResidualUnet_3Dense_5Kernels import Residual_Unet_3D_5K



In [13]:
fixed_images_train = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/fixed_images_train.npy')
fixed_images_test = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/fixed_images_test.npy')
moving_images_train = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/moving_images_train.npy')
moving_images_test = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/moving_images_test.npy')
fixed_images_valid = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/fixed_images_valid.npy')
moving_images_valid = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/moving_images_valid.npy')
y_train = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/y_train.npy')
y_test = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/y_test.npy')
y_valid = np.load('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Code/FrameWork/Loaded_data/y_valid.npy')

In [23]:
model = tf.keras.models.load_model("/Users/ahmed_ali/Downloads/Unet_3Dense_5Kernel_with_mask.keras", custom_objects={'MaskLoss': MaskLoss, 'MAELoss': MAELoss, 'Residual_Unet_1D_7K': Residual_Unet_1D_7K})

In [ ]:
model = tf.keras.models.load_model("/Users/ahmed_ali/Downloads/Unet_3Dense_5Kernel_with_mask.keras", custom_objects={'MaskLoss': MaskLoss, 'MAELoss': MAELoss, 'Residual_Unet_1D_7K': Residual_Unet_1D_7K})
/Users/ahmed_ali/Downloads/Unet_3Dense_5Kernel_with_mask.keras

In [24]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))


Num GPUs Available:  1


In [25]:
batch_size = 32
num_epochs = 2

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=1.0)
fixed_input = Input(shape=(128, 128, 1), name="fixed_image")
moving_input = Input(shape=(128, 128, 1), name="moving_image")
modelnet = model
# model_name = model.__class__.__name__
# self.model_names.append(model_name)
out_def = modelnet([moving_input, fixed_input])
model = Model(inputs=[moving_input, fixed_input], outputs=out_def)

model.compile(optimizer=optimizer, loss=MaskLoss(), metrics=[MAELoss()])

# file_name = f"{model_name}.keras"  
check_point_path = '/Users/ahmed_ali/Downloads/Unet_3Dense_5Kernel_with_mask.keras'
# Create an empty .keras file
open(check_point_path, "w").close()


# Define callbacks
checkpoint_callback = ModelCheckpoint(
check_point_path,
monitor='val_loss',
save_best_only=True,
mode='min',
verbose=1

)

stopping_callback = tf.keras.callbacks.EarlyStopping(
monitor='val_loss',
patience=15,
mode='min',
verbose=1
)
with tf.device('/GPU:0'):
    history = model.fit(
    [moving_images_train, fixed_images_train],
    y_train,
    batch_size=batch_size,
    epochs = num_epochs,
    validation_data=([moving_images_valid, fixed_images_valid], y_valid),
    callbacks=[checkpoint_callback, stopping_callback],
    )


Epoch 1/2
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.0635 - mae_loss_2: 1.1460
Epoch 1: val_loss improved from inf to 0.00473, saving model to /Users/ahmed_ali/Downloads/Unet_3Dense_5Kernel_with_mask.keras
198/198 ━━━━━━━━━━━━━━━━━━━━ 682s 3s/step - loss: 0.0634 - mae_loss_2: 1.1434 - val_loss: 0.0047 - val_mae_loss_2: 0.2853
Epoch 2/2
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0054 - mae_loss_2: 0.2541
Epoch 2: val_loss did not improve from 0.00473
198/198 ━━━━━━━━━━━━━━━━━━━━ 902s 5s/step - loss: 0.0054 - mae_loss_2: 0.2540 - val_loss: 0.0071 - val_mae_loss_2: 0.2188
